In [1]:
#q1
with open('/etc/os-release', 'r') as f:
    osRelease = f.read()
osRelease

'PRETTY_NAME="Ubuntu 24.04.2 LTS"\nNAME="Ubuntu"\nVERSION_ID="24.04"\nVERSION="24.04.2 LTS (Noble Numbat)"\nVERSION_CODENAME=noble\nID=ubuntu\nID_LIKE=debian\nHOME_URL="https://www.ubuntu.com/"\nSUPPORT_URL="https://help.ubuntu.com/"\nBUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"\nPRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"\nUBUNTU_CODENAME=noble\nLOGO=ubuntu-logo\n'

In [2]:
import subprocess

In [3]:
#q2
result = str(subprocess.run(['lscpu'], capture_output=True, text=True))
result

"CompletedProcess(args=['lscpu'], returncode=0, stdout='Architecture:                         x86_64\\nCPU op-mode(s):                       32-bit, 64-bit\\nAddress sizes:                        46 bits physical, 48 bits virtual\\nByte Order:                           Little Endian\\nCPU(s):                               2\\nOn-line CPU(s) list:                  0,1\\nVendor ID:                            GenuineIntel\\nModel name:                           Intel(R) Xeon(R) CPU @ 2.20GHz\\nCPU family:                           6\\nModel:                                79\\nThread(s) per core:                   2\\nCore(s) per socket:                   1\\nSocket(s):                            1\\nStepping:                             0\\nBogoMIPS:                             4399.99\\nFlags:                                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo

In [4]:
from pyarrow import fs

In [5]:
gcs = fs.GcsFileSystem()
with open("../wi-schools-raw.parquet", "rb") as f:
    data = f.read()
with gcs.open_output_stream("cs544p8bucket/wi-schools-raw.parquet") as stream:    
    stream.write(data)


In [6]:
#q3
selector = fs.FileSelector('cs544p8bucket', recursive=True)
paths = []
info = gcs.get_file_info(selector)
for file in info:
    paths.append(str(file.path))
paths

['cs544p8bucket/wi-schools-raw.parquet']

In [7]:
#q4
info[0].mtime_ns

1745979591590000000

In [8]:
from google.cloud import dataform
client = dataform.DataformClient()

sqlx = """
config {
  type: "table",
  name: "wi_counties",
}

SELECT *
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = "55"

"""

client.write_file(dataform.WriteFileRequest(
    workspace="projects/still-protocol-457514-r0/locations/us-central1/repositories/p8dataform/workspaces/mainp8",
    path="definitions/wi_counties.sqlx",
    contents=bytes(sqlx, "utf-8")
))



In [9]:
sqlx = """
config {
  type: "operations",
  name: "schools",
  hasOutput: true,
}

LOAD DATA OVERWRITE still-protocol-457514-r0.p8.schools
FROM FILES(
    uris=["gs://cs544p8bucket/wi-schools-raw.parquet"],
    format="parquet"
)


"""

client.write_file(dataform.WriteFileRequest(
    workspace="projects/still-protocol-457514-r0/locations/us-central1/repositories/p8dataform/workspaces/mainp8",
    path="definitions/schools.sqlx",
    contents=bytes(sqlx, "utf-8")
))

In [10]:
sqlx = """
config {
  type: "table",
  name: "wi_county_schools",
}

SELECT s.*, c.county_name, ST_GEOGPOINT(s.longitude, s.latitude) AS location
FROM ${ref('schools')} AS s
CROSS JOIN ${ref('wi_counties')} AS c
WHERE ST_WITHIN(ST_GEOGPOINT(s.longitude, s.latitude), c.county_geom) 

"""

client.write_file(dataform.WriteFileRequest(
    workspace="projects/still-protocol-457514-r0/locations/us-central1/repositories/p8dataform/workspaces/mainp8",
    path="definitions/wi_county_schools.sqlx",
    contents=bytes(sqlx, "utf-8")
))

In [11]:
#used chatgpt, pdf of output in gitlab (p1)
project_id = "still-protocol-457514-r0"
location = "us-central1"
repository_id = "p8dataform"
workspace_id = "mainp8"

parent = client.repository_path(project=project_id, location=location, repository=repository_id)
workspace_path = client.workspace_path(
    project=project_id,
    location=location,
    repository=repository_id,
    workspace=workspace_id,
)
compilation_result = dataform.CompilationResult(
    workspace = workspace_path,
)

compilation_result_name = client.create_compilation_result(
    parent=parent,
    compilation_result=compilation_result,
)

In [12]:
%load_ext bigquery_magics

In [13]:
from google.cloud import bigquery
bq = bigquery.Client()

In [14]:
#q5
df = bq.query("""
select count(*)
from p8.wi_counties
""").to_dataframe()
ans = int(df.iloc[0,0])
ans

72

In [16]:
#q6
df = bq.query("""
select count(*)
from p8.wi_county_schools as s 
where s.agency_type = "Public school"
""").to_dataframe()
ans = int(df.iloc[0,0])
ans

2116

In [17]:
#q7
response = client.query_compilation_result_actions(
    request={"name": compilation_result_name.name}
)
ans = {}
for act in response.compilation_result_actions:
    if act.target.name not in ans:
        ans[act.target.name] = []
        for t in act.relation.dependency_targets:
            ans[act.target.name].append(t.name)
ans

{'first_view': [],
 'schools': [],
 'second_view': ['first_view'],
 'wi_counties': [],
 'wi_county_schools': ['schools', 'wi_counties']}

In [18]:
config = bigquery.QueryJobConfig(use_query_cache=False)
q = bq.query("""
    SELECT schools.county_name, COUNT(*)
    FROM p8.wi_county_schools as schools
    GROUP BY schools.county_name
    """, job_config=config)
df = q.to_dataframe()

In [19]:
#q8
import pandas as pd
#used google ai, searched to dict pandas column as key\n"
pd.Series(df['f0_'].values, index=df['county_name']).to_dict()

{'Brown': 123,
 'Shawano': 29,
 'Price': 9,
 'Waukesha': 177,
 'Kenosha': 56,
 'Milwaukee': 420,
 'Racine': 68,
 'Rock': 72,
 'Dane': 217,
 'Portage': 38,
 'Fond du Lac': 55,
 'Ozaukee': 38,
 'Calumet': 30,
 'Washington': 56,
 'Dodge': 51,
 'Jefferson': 52,
 'Manitowoc': 44,
 'Sheboygan': 63,
 'Walworth': 56,
 'Green': 23,
 'Lafayette': 18,
 'Iowa': 16,
 'Sauk': 42,
 'Richland': 14,
 'Grant': 39,
 'Crawford': 17,
 'Columbia': 41,
 'Green Lake': 15,
 'Juneau': 26,
 'Marquette': 10,
 'Pierce': 29,
 'St. Croix': 35,
 'Polk': 28,
 'Marinette': 29,
 'Outagamie': 96,
 'Oconto': 20,
 'Kewaunee': 15,
 'Door': 18,
 'Marathon': 72,
 'Langlade': 13,
 'Wood': 47,
 'Clark': 38,
 'Taylor': 13,
 'Lincoln': 16,
 'Oneida': 17,
 'Vilas': 13,
 'La Crosse': 59,
 'Trempealeau': 27,
 'Vernon': 29,
 'Monroe': 28,
 'Buffalo': 11,
 'Eau Claire': 54,
 'Chippewa': 35,
 'Dunn': 22,
 'Pepin': 7,
 'Jackson': 10,
 'Barron': 40,
 'Washburn': 16,
 'Ashland': 12,
 'Sawyer': 13,
 'Rusk': 11,
 'Douglas': 18,
 'Winnebago'

In [20]:
#q9
q.total_bytes_billed / 1024**2 
import math
TB = 1024**4
math.floor(TB / q.total_bytes_billed)

104857

In [21]:
%%bigquery df
#used chatgpt, output in gitlab (p2), and https://stackoverflow.com/questions/40381322/with-tablename-as-equivalent-in-oracle
with mid as (
    SELECT s.school_name, s.school_type, s.county_name, s.location, s.agency_type
    FROM p8.wi_county_schools as s
    WHERE school_type = "Middle School" AND county_name = "Dane" AND agency_type = "Public school"
),
    high as (
        SELECT s.school_name, s.school_type, s.county_name, s.location, s.agency_type
        FROM p8.wi_county_schools as s
        WHERE school_type = "High School" AND county_name = "Dane" AND agency_type = "Public school"
    )
    SELECT m.school_name, MIN_BY(h.school_name, ST_DISTANCE(m.location, h.location)) AS closest_high_school
    FROM mid as m
    CROSS JOIN high as h
    GROUP BY m.school_name
    ORDER BY m.school_name

Query is running:   0%|          |

Downloading:   0%|          |

In [22]:
#q10
ans = dict(zip(df['school_name'], df['closest_high_school']))
ans

{'Badger Ridge Middle': 'Verona Area High',
 'Badger Rock Middle': 'West High',
 'Belleville Middle': 'Belleville High',
 'Black Hawk Middle': 'Shabazz-City High',
 'Central Heights Middle': 'Prairie Phoenix Academy',
 'Cherokee Heights Middle': 'Capital High',
 'De Forest Middle': 'De Forest High',
 'Deerfield Middle': 'Deerfield High',
 'Ezekiel Gillespie Middle School': 'Vel Phillips Memorial High School',
 'Glacial Drumlin School': 'LaFollette High',
 'Glacier Creek Middle': 'Middleton High',
 'Hamilton Middle': 'Capital High',
 'Indian Mound Middle': 'McFarland High',
 'Innovative and Alternative Middle': 'Innovative High',
 'James Wright Middle': 'West High',
 'Kromrey Middle': 'Middleton High',
 'Marshall Middle': 'Marshall High',
 'Mount Horeb Middle': 'Mount Horeb High',
 'Nikolay Middle': 'Koshkonong Trails School',
 "O'Keeffe Middle": 'Innovative High',
 'Oregon Middle': 'Oregon High',
 'Patrick Marsh Middle': 'Prairie Phoenix Academy',
 'Prairie View Middle': 'Sun Prairie W